In [40]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Split Data

In [41]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                             ])

# Train Model

In [17]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [42]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

In [43]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [48]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = Net()

In [49]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)

In [50]:
# CPU only training
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [51]:
for epoch in range(1, 20):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 2.299830
Train Epoch: 1 [640/3000 (21%)]	Loss: 2.296133
Train Epoch: 1 [1280/3000 (43%)]	Loss: 2.300455
Train Epoch: 1 [1920/3000 (64%)]	Loss: 2.296795
Train Epoch: 1 [2560/3000 (85%)]	Loss: 2.278701

Test set: Average loss: 2.2544, Accuracy: 3264/10000 (33%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 2.249753
Train Epoch: 2 [640/3000 (21%)]	Loss: 2.240208
Train Epoch: 2 [1280/3000 (43%)]	Loss: 2.230728
Train Epoch: 2 [1920/3000 (64%)]	Loss: 2.052393
Train Epoch: 2 [2560/3000 (85%)]	Loss: 2.059448

Test set: Average loss: 1.9747, Accuracy: 3946/10000 (39%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 2.086519
Train Epoch: 3 [640/3000 (21%)]	Loss: 1.792334
Train Epoch: 3 [1280/3000 (43%)]	Loss: 1.634648
Train Epoch: 3 [1920/3000 (64%)]	Loss: 1.703681
Train Epoch: 3 [2560/3000 (85%)]	Loss: 1.300854

Test set: Average loss: 1.0068, Accuracy: 7291/10000 (73%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 1.368964
Train Epoch: 4 [640/3000 (21%)]	Loss: 1.424469
Train Epoch: 4 [1280/

# Create Sample Submission

In [59]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [60]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [61]:
test(1, test_loader)


Test set: Average loss: 0.1253, Accuracy: 9586/10000 (96%)



In [62]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [63]:
label_predict

array([ 7.,  2.,  1., ...,  4.,  5.,  6.])

In [64]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [65]:
predict_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [68]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
